So the data is gathered from trading view and added to use as a analysis point

Data of 15min charts is used for alert signs and the data of 1min charts is used to check accurate Target profit points and stop loss points.

Will ignore all datasets for privacy.


In [ ]:
import pandas as pd

# Load the CSV files
minute_data_c23400 = pd.read_csv('/mnt/data/NSE_NIFTY240620C23400, 1.csv')
minute_data_15_c23400 = pd.read_csv('/mnt/data/NSE_NIFTY240620C23400, 15.csv')

# Filter buy signals
def filter_signals(data):
    buy_signals = data[(data['Original Buy Signal'] == True) & (data['Short Signal'] == False)]
    return buy_signals

buy_signals = filter_signals(minute_data_15_c23400)

# Simulate trades
def simulate_trades(buy_signals, minute_data, profit_target=0.02, stop_loss=0.01, min_stop_loss=0.005):
    results = []
    for index, buy_signal in buy_signals.iterrows():
        entry_price = buy_signal['close']
        entry_time = buy_signal['time']
        profit_price = entry_price * (1 + profit_target)
        stop_price = entry_price * (1 - max(stop_loss, min_stop_loss))  # Ensure stop loss is at least min_stop_loss
        
        trade_result = {
            'entry_time': entry_time,
            'entry_price': entry_price,
            'exit_time': None,
            'exit_price': None,
            'profit': None
        }
        
        subsequent_data = minute_data[minute_data['time'] > entry_time]
        for _, row in subsequent_data.iterrows():
            if row['high'] >= profit_price:
                trade_result['exit_time'] = row['time']
                trade_result['exit_price'] = profit_price
                trade_result['profit'] = profit_price - entry_price
                break
            if row['low'] <= stop_price:
                trade_result['exit_time'] = row['time']
                trade_result['exit_price'] = stop_price
                trade_result['profit'] = stop_price - entry_price
                break
        
        results.append(trade_result)
    
    return pd.DataFrame(results)



Analye the results now

In [ ]:

# Analyze different profit and stop loss percentages
def analyze_profit_and_stop_loss(buy_signals, minute_data, profit_percentages, stop_loss_percentages, min_stop_loss=0.005):
    analysis_results = []
    for profit_target in profit_percentages:
        for stop_loss in stop_loss_percentages:
            trade_results = simulate_trades(buy_signals, minute_data, profit_target, stop_loss, min_stop_loss)
            total_profit = trade_results['profit'].sum()
            analysis_results.append((profit_target, stop_loss, total_profit))
    return pd.DataFrame(analysis_results, columns=['Profit Target', 'Stop Loss', 'Total Profit'])

# Define profit percentages and stop loss percentages to analyze
profit_percentages = [i/100 for i in range(1, 6)]
stop_loss_percentages = [i/100 for i in range(1, 6)]

# Perform the analysis
analysis_results = analyze_profit_and_stop_loss(buy_signals, minute_data_c23400, profit_percentages, stop_loss_percentages)


Store them as CSV and printout for visulaisation

In [ ]:
# Save the results to a CSV file
analysis_results.to_csv('/mnt/data/profit_stop_loss_analysis.csv', index=False)

import ace_tools as tools; tools.display_dataframe_to_user(name="Profit and Stop Loss Analysis", dataframe=analysis_results)
